## How to use UpTrain with LlamaIndex

**Overview**: In this example, we will see how to use UpTrain with LlamaIndex. 

**Problem**: There are two main problems:
1. The data that most Large Language Models are trained on is not representative of the data that they are used on. This leads to a mismatch between the training and test distributions, which can lead to poor performance. 
2. The results generated by Large Language Models are not always reliable. The responses might not be relevant to the prompt, not align with the desired tone or the context, or might be offensive etc.

**Solution**: The above two problems are solved by two different tools and we will show you how to use them together:
1. LlamaIndex solves the first problem by allowing you to perform Retrieval Augmented Generation (RAG) with a retriever that is fine-tuned on your own data. This allows you to use your own data to fine-tune a retriever, and then use that retriever to perform RAG.
2. UpTrain solves the second problem by allowing you to perform evaluations on the generated responses. This helps you to ensure that the responses are relevant to the prompt, align with the desired tone or the context, and are not offensive etc.

## Install UpTrain with all dependencies

```bash
pip install uptrain
uptrain-add --feature full
```

## Import required libraries

In [1]:
import os
import openai 
import pandas as pd

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from uptrain import EvalLLM, Evals, EvalLlamaIndex, Settings

**This notebook uses the OpenAI API to generate text for prompts as well as to create the Vector Store Index. So, set openai.api_key and create an EvalLLM client using your OpenAI API key**

In [2]:
openai.api_key = 'sk-*************' # your openai api key
settings = Settings(openai_api_key=openai.api_key)
eval_llm = EvalLLM(settings)

## Create the datset folder for the query engine

You can use any documents that you have to do thid. For this tutorial, we will use data on New York City extracted from wikipedia. We will only add one document to the folder, but you can add as many as you want.

In [3]:
url = "https://uptrain-assets.s3.ap-south-1.amazonaws.com/data/nyc_text.txt"
if not os.path.exists('nyc_wikipedia'):
   os.makedirs('nyc_wikipedia')
dataset_path = os.path.join('./nyc_wikipedia', "nyc_text.txt")

if not os.path.exists(dataset_path):
    import httpx
    r = httpx.get(url)
    with open(dataset_path, "wb") as f:
        f.write(r.content)

## Create a query engine using LlamaIndex

Let's create a vector store index using LLamaIndex and then use that as a query engine to retrieve relevant sections from the documentation.

In [4]:
documents = SimpleDirectoryReader("./nyc_wikipedia/").load_data()

vector_index = VectorStoreIndex.from_documents(
    documents, service_context=ServiceContext.from_defaults(chunk_size=512)
)

query_engine = vector_index.as_query_engine()

## Create the EvalLlamaIndex object

Now that we have created the query engine, we can use it to create an EvalLlamaIndex object. This object will be used to generate responses for the queries.

In [5]:
llamaindex_object = EvalLlamaIndex(query_engine=query_engine)

## Make the list of queries

Before we can generate responses, we need to create a list of queries. Since the query engine is trained on New York City, we will create a list of queries related to New York City.

In [6]:
data = [
    {"question": "What is the population of New York City?"},
    {"question": "What is the area of New York City?"},
    {"question": "What is the largest borough in New York City?"},
    {"question": "What is the average temperature in New York City?"},
    {"question": "What is the main airport in New York City?"},
    {"question": "What is the famous landmark in New York City?"},
    {"question": "What is the official language of New York City?"},
    {"question": "What is the currency used in New York City?"},
    {"question": "What is the time zone of New York City?"},
    {"question": "What is the famous sports team in New York City?"}
]

## Perform the evaluation

Now that we have the list of queries, we can use the EvalLlamaIndex object to generate responses for the queries and then perform evaluations on the responses. You can find an extensive list of the evaluations offered by UpTrain [here](https://docs.uptrain.ai/key-components/evals). We have chosen two that we found to be the most relevant for this tutorial:

1. **Context Relevance**: This evaluation checks whether the retrieved context is relevant to the query. This is important because the retrieved context is used to generate the response. If the retrieved context is not relevant to the query, then the response will not be relevant to the query either.

2. **Response Conciseness**: This evaluation checks whether the response is concise. This is important because the response should be concise and should not contain any unnecessary information.


In [7]:
results = llamaindex_object.evaluate(eval_llm, 
                              data=data, 
                              checks=[
                                   Evals.CONTEXT_RELEVANCE,
                                   Evals.RESPONSE_CONCISENESS 
                                   ]
                             )

2023-11-22 19:57:25.734 | INFO     | uptrain.framework.evalllm:evaluate:98 - Sending evaluation request for rows 0 to <50 to the Uptrain
2023-11-22 19:57:37.411 | INFO     | uptrain.framework.evalllm:evaluate:98 - Sending evaluation request for rows 0 to <50 to the Uptrain
2023-11-22 19:57:48.561 | INFO     | uptrain.framework.evalllm:evaluate:98 - Sending evaluation request for rows 0 to <50 to the Uptrain


In [8]:
results

[{'question': 'What is the population of New York City?',
  'response': '\nThe population of New York City in 2020 was 8,804,190.',
  'context': "Aeromedical Staging Squadron, and a military entrance processing station. Other formerly active military reservations still used for National Guard and military training or reserve operations in the city include Fort Wadsworth in Staten Island and Fort Totten in Queens.\n\n\n== Demographics ==\n\nNew York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over